In [1]:
import numpy as np
import tensorflow as tf
from dct_funcs import *
np.random.seed(73)

In [2]:
input_image_size = 8
NC = 1
print("input_image_size: " + str(input_image_size))

coef_cos = prep_coef_cos(np.float32, Nd=8)
print("coef_cos.shape: " + str(coef_cos.shape))
print("coef_cos: \n" + str(coef_cos) + '\n')

im_c = np.random.randint(0,255,[8,8])

n = 8
stride_size = n
type1=np.float32

dct_kern = np.zeros([n,n,1,64])
dct_kern = dct_kern.astype(type1)

input_image_size: 8
coef_cos.shape: (8, 8)
coef_cos: 
[[ 1.          1.          1.          1.          1.          1.
   1.          1.        ]
 [ 0.98078528  0.83146961  0.55557023  0.19509032 -0.19509032 -0.55557023
  -0.83146961 -0.98078528]
 [ 0.92387953  0.38268343 -0.38268343 -0.92387953 -0.92387953 -0.38268343
   0.38268343  0.92387953]
 [ 0.83146961 -0.19509032 -0.98078528 -0.55557023  0.55557023  0.98078528
   0.19509032 -0.83146961]
 [ 0.70710678 -0.70710678 -0.70710678  0.70710678  0.70710678 -0.70710678
  -0.70710678  0.70710678]
 [ 0.55557023 -0.98078528  0.19509032  0.83146961 -0.83146961 -0.19509032
   0.98078528 -0.55557023]
 [ 0.38268343 -0.92387953  0.92387953 -0.38268343 -0.38268343  0.92387953
  -0.92387953  0.38268343]
 [ 0.19509032 -0.55557023  0.83146961 -0.98078528  0.98078528 -0.83146961
   0.55557023 -0.19509032]]



In [3]:
i_n =0
for iu in range(n):
    for iv in range(n):
        dctW = dct_W(coef_cos, iu, iv, type1, type1)
        dct_kern[:,:,0,i_n] = dctW[:,:,0,0]
        i_n = i_n + 1

#Conv2D kernel
compW = tf.constant(dct_kern)
print("\ncompW: " + str(compW))


compW: Tensor("Const:0", shape=(8, 8, 1, 64), dtype=float32)


In [4]:
## input layer
with tf.name_scope('DATA'):
    Data_in = tf.placeholder(tf.float32,shape=[None,input_image_size,input_image_size,NC])
    print(Data_in)


with tf.name_scope('Compress_Conv'):
    comp = tf.nn.conv2d(Data_in, compW, strides=[1, stride_size, stride_size, 1], padding='VALID')
    print(comp)

Tensor("DATA/Placeholder:0", shape=(?, 8, 8, 1), dtype=float32)
Tensor("Compress_Conv/Conv2D:0", shape=(?, 1, 1, 64), dtype=float32)


In [5]:
# Put image in proper format
data_in = np.zeros([1,input_image_size,input_image_size,NC])
data_in[0,:,:,0] = im_c

In [6]:
#Run example compression:
with tf.Session() as sess:
    comp_o=sess.run(comp,feed_dict={Data_in:data_in})

In [7]:
#Check output:
print("\ncomp_o shape: " + str(comp_o.shape))
print("\ncomp_o: \n" + str(comp_o))


comp_o shape: (1, 1, 1, 64)

comp_o: 
[[[[ 9.37499939e+02  9.82029953e+01  7.94020691e+01  6.68032532e+01
    -4.07500000e+01  8.61678009e+01  9.35352325e-01 -1.64789932e+02
     2.04412079e+00  1.22867027e+02 -1.14696205e+02 -1.61257896e+01
     6.38304977e+01  6.71096611e+00  8.94229889e-01  2.76039562e+01
     8.35222321e+01 -1.32541962e+02  4.48975906e+01 -9.61036682e+00
     1.75434113e+01  3.80550766e+01  3.11038017e+01 -8.01738663e+01
    -6.42647552e+01  1.58439392e+02  5.27777405e+01 -6.43810883e+01
    -7.13396301e+01 -2.42038517e+01  6.53008270e+01 -2.52245598e+01
    -7.49999847e+01 -1.21392952e+02 -3.99551849e+01 -2.38254375e+01
    -1.57750000e+02  4.27300358e+00  1.34801331e+02  7.62428131e+01
     6.43402863e+00  5.35582008e+01  2.63879929e+01  4.28410721e+00
    -3.22338104e+01  1.12583447e+01 -1.05793221e+02  7.57403469e+00
    -3.81138153e+01  7.10592651e+01  2.93537998e+01 -1.62391701e+01
     5.28060608e+01 -3.48407173e+01 -4.91475906e+01  3.75562477e+01
     3.14

In [8]:
to2d_kern = np.zeros([n,n,1,64])
to2d_kern = to2d_kern.astype(type1)
i_n =0
for iu in range(n):
    for iv in range(n):
        to2d_kern[iu,iv,0,i_n] = 1.0
        i_n = i_n + 1
        
#Conv2D kernel
to2dW = tf.constant(to2d_kern)
print("to2dW: " + str(to2dW))

to2dW: Tensor("Const_1:0", shape=(8, 8, 1, 64), dtype=float32)


In [9]:
to2d_kern[...,0,1]

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [10]:
Dc = tf.nn.conv2d_transpose(comp,to2dW,[1,n,n,1],strides=[1,n,n,1],padding='VALID')
print('Dc: ' + str(Dc))

Dc: Tensor("conv2d_transpose:0", shape=(1, 8, 8, 1), dtype=float32)


In [11]:
#Run example:
with tf.Session() as sess:
    Dc_o=sess.run(Dc,feed_dict={Data_in:data_in})
    
Dc_2d = Dc_o[0,:,:,0]
print("Dc_o shape: " + str(Dc_o.shape))
#print("\nDc_o: \n" + str(Dc_o))
print("\nDc_2d: \n" + str(Dc_2d))

Dc_o shape: (1, 8, 8, 1)

Dc_2d: 
[[ 9.37499939e+02  9.82029419e+01  7.94020615e+01  6.68032532e+01
  -4.07500000e+01  8.61678009e+01  9.35342729e-01 -1.64789917e+02]
 [ 2.04410362e+00  1.22867004e+02 -1.14696198e+02 -1.61257896e+01
   6.38305016e+01  6.71096277e+00  8.94242287e-01  2.76039562e+01]
 [ 8.35222244e+01 -1.32541977e+02  4.48975983e+01 -9.61035728e+00
   1.75434132e+01  3.80550652e+01  3.11038189e+01 -8.01738815e+01]
 [-6.42647705e+01  1.58439392e+02  5.27777367e+01 -6.43810730e+01
  -7.13396149e+01 -2.42038460e+01  6.53008041e+01 -2.52245674e+01]
 [-7.50000000e+01 -1.21392960e+02 -3.99551697e+01 -2.38254509e+01
  -1.57750000e+02  4.27300072e+00  1.34801315e+02  7.62428131e+01]
 [ 6.43403625e+00  5.35582199e+01  2.63879852e+01  4.28410435e+00
  -3.22338219e+01  1.12583485e+01 -1.05793213e+02  7.57401371e+00]
 [-3.81138153e+01  7.10592499e+01  2.93538303e+01 -1.62391853e+01
   5.28060455e+01 -3.48407173e+01 -4.91476097e+01  3.75562515e+01]
 [ 3.14497585e+01  1.21091904e+02 -

In [12]:
from scipy import fftpack
def dct_2d(image):
    return fftpack.dct(fftpack.dct(image.T, norm='ortho').T, norm='ortho')

In [13]:
dc_2d = dct_2d(im_c)
print('dc_2d:\n' + str(dc_2d))

dc_2d:
[[ 9.37500000e+02  9.82030198e+01  7.94020631e+01  6.68032652e+01
  -4.07500000e+01  8.61678069e+01  9.35344828e-01 -1.64789941e+02]
 [ 2.04407907e+00  1.22867017e+02 -1.14696206e+02 -1.61257934e+01
   6.38304972e+01  6.71095521e+00  8.94239831e-01  2.76039422e+01]
 [ 8.35222260e+01 -1.32541979e+02  4.48976009e+01 -9.61036148e+00
   1.75434124e+01  3.80550754e+01  3.11038149e+01 -8.01738692e+01]
 [-6.42647758e+01  1.58439407e+02  5.27777564e+01 -6.43810868e+01
  -7.13396231e+01 -2.42038478e+01  6.53008262e+01 -2.52245732e+01]
 [-7.50000000e+01 -1.21392949e+02 -3.99551736e+01 -2.38254492e+01
  -1.57750000e+02  4.27300404e+00  1.34801323e+02  7.62428113e+01]
 [ 6.43403476e+00  5.35582041e+01  2.63879911e+01  4.28411209e+00
  -3.22338093e+01  1.12583506e+01 -1.05793228e+02  7.57402967e+00]
 [-3.81138134e+01  7.10592543e+01  2.93538149e+01 -1.62391866e+01
   5.28060478e+01 -3.48407195e+01 -4.91476009e+01  3.75562520e+01]
 [ 3.14497778e+01  1.21091902e+02 -3.80394567e+01  9.65442166e

In [14]:
mse = np.mean(np.square(dc_2d-Dc_2d))
print('mse= ' + str(mse))

dmax = np.max(np.abs(dc_2d-Dc_2d))
print('dmax= ' + str(dmax))

mse= 2.398477636428144e-10
dmax= 7.785823599704145e-05


In [15]:
idct_kern = np.zeros([n,n,1,64])
idct_kern = idct_kern.astype(type1)

i_n =0
for iu in range(n):
    for iv in range(n):
        idctW = idct_W(coef_cos, iu, iv, type1, type1)
        idct_kern[:,:,0,i_n] = idctW[:,:,0,0]
        i_n = i_n + 1

#Conv2D kernel
icompW = tf.constant(idct_kern)
print("icompW: " + str(icompW))

icompW: Tensor("Const_2:0", shape=(8, 8, 1, 64), dtype=float32)


In [16]:
with tf.name_scope('Compress_Conv'):
    icomp = tf.nn.conv2d(Data_in, icompW, strides=[1, stride_size, stride_size, 1], padding='VALID')
    print(icomp)
    
Dic = tf.nn.conv2d_transpose(icomp,to2dW,[1,n,n,1],strides=[1,n,n,1],padding='VALID')
print('Dic: ' + str(Dic))

Tensor("Compress_Conv_1/Conv2D:0", shape=(?, 1, 1, 64), dtype=float32)
Dic: Tensor("conv2d_transpose_1:0", shape=(1, 8, 8, 1), dtype=float32)


In [17]:
im_c

array([[150, 146, 174, 138,  16, 136,  67,  65],
       [166, 106,  71, 225, 100, 181, 110, 135],
       [247, 180,  43, 233, 181,  46,  67,  50],
       [ 62, 116, 100,  67,  85, 129, 191,  39],
       [143, 104, 203,  73,  27, 169,  12,  28],
       [189, 225,  61,  94,  10,  44,  75,  90],
       [209, 211,  46, 181,  22, 164, 161, 127],
       [ 72,  39,  53,  86,  47, 253, 230, 230]])

In [18]:
data_dct = np.zeros([1,input_image_size,input_image_size,NC])
data_dct[0,:,:,0] = Dc_2d
with tf.Session() as sess:
    Dic_o=sess.run(Dic,feed_dict={Data_in:data_dct})
    
Dic_2d = Dic_o[0,:,:,0]
print("Dic_o shape: " + str(Dic_o.shape))
#print("\nDc_o: \n" + str(Dc_o))
print("\nDic_2d: \n" + str(Dic_2d))

Dic_o shape: (1, 8, 8, 1)

Dic_2d: 
[[150.00002  145.99994  173.99997  137.99998   16.00002  136.
   66.99999   65.000015]
 [165.99995  105.99994   70.999985 224.99997  100.00002  181.00003
  110.000015 135.00002 ]
 [246.99997  179.99995   43.000004 233.00002  180.99998   45.999992
   67.00004   50.000027]
 [ 61.999992 115.99997  100.000015  66.99999   84.99997  129.00003
  191.00002   39.00004 ]
 [142.99997  103.99992  202.99997   72.99999   27.00001  168.99994
   12.000029  28.000025]
 [188.99994  224.99995   60.999996  93.99997   10.00002   43.99997
   75.        90.00001 ]
 [209.00002  210.99995   45.999977 180.99998   21.999992 164.00009
  160.99994  127.00002 ]
 [ 71.99997   38.999966  52.999992  85.99998   46.999992 252.99992
  229.99994  230.      ]]


In [20]:
mse = np.mean(np.square(Dic_2d-im_c))
print('mse= ' + str(mse))

mse = np.mean(np.square(np.round(Dic_2d)-im_c))
print('mse round= ' + str(mse))

dmax = np.max(np.abs(Dic_2d-im_c))
print('dmax= ' + str(dmax))

mse= 1.1635421515165945e-09
mse round= 0.0
dmax= 9.1552734375e-05
